In [144]:
import pandas as pd
import numpy as np

import gensim
from gensim import corpora, models
from gensim.models import CoherenceModel

import time
import pickle
from tqdm import tqdm

import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from konlpy.tag import Twitter

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [71]:
with open('artNouns_final.dat','rb') as file: # load
    nouns = pickle.load(file)

In [2]:
# with open('artAd_nouns_final.dat','rb') as file: # load
#     ad_nouns = pickle.load(file)

nouns를 token으로 사용한다

전처리해주자 

TFIDF 기준으로 1.5 이하 단어는 불용어처리

In [72]:
with open('TFIDF_nouns_final.dic','rb') as file:
    TFIDF = pickle.load(file)

In [6]:
TFIDF[1]

{'가장': 1.110497112026965,
 '가지': 0.8472257461185791,
 '감': 1.3416919836089962,
 '개최': 1.6472856306569452,
 '건물': 2.465261985167852,
 '것': 0.13410400781201817,
 '결실': 3.6582446305516974,
 '경이': 2.9705667858053655,
 '경험': 1.0376612665698866,
 '공간': 0.9398263662103566,
 '공감': 2.014007815344043,
 '공개': 2.8191968525978126,
 '공사': 3.4895808945229687,
 '공유': 2.016641346711018,
 '공존': 1.8749676971007725,
 '과거': 1.388094111007229,
 '과정': 0.9622435263454289,
 '관계': 1.124769983124599,
 '관람객': 1.9283580100538265,
 '관상': 4.818977824883205,
 '관심': 1.3804849928471372,
 '권': 3.1976554673057973,
 '그': 0.14326591797223653,
 '그동안': 2.5572012612067687,
 '그룹': 2.4931275451104704,
 '그림자': 2.2358195231749436,
 '긍정': 2.2987526817332506,
 '기': 1.9549880860344273,
 '기반': 2.137814908214764,
 '기법': 1.5346288702102806,
 '기억': 1.7510554505558265,
 '기와집': 5.071665215133719,
 '기이': 2.9937227938892716,
 '기회': 1.6217188333967152,
 '길': 1.5930565043617895,
 '김유정': 5.983969181170632,
 '김희연': 6.312461858352128,
 '깊이': 1.8

In [73]:
nouns_refined = []
for i, doc in enumerate(nouns):
    tmp = []
    for token in doc:
        try:
            if (TFIDF[i][token] >1.5):
                tmp.append(token)
        except Exception as e:
            print(e)
    nouns_refined.append(tmp)

In [74]:
dic = corpora.Dictionary(nouns_refined)

In [117]:
# with open('nouns_refined_final.dic','wb') as file:
#     pickle.dump(dic,file)

In [114]:
with open('nouns_refined_final.dic','rb') as file:
    dic = pickle.load(file)

In [115]:
type(dic)

gensim.corpora.dictionary.Dictionary

In [116]:
len(dic) #출현 단어 개수

62349

In [77]:
corpus = [dic.doc2bow(doc) for doc in nouns_refined] #단어별 출현 횟수 vector 구성

In [118]:
with open('corpus_nouns_refined_final.lst','wb') as file:
    pickle.dump(corpus,file)

In [78]:
%%time
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus, 
                        num_topics=5, 
                        id2word = dic, 
                        passes=20
                        ) #LDA model 생성

Wall time: 5min 42s


In [79]:
ldamodel.save('lda_exhb_5.model')

In [84]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda_exhb_10.model')

In [90]:
len(corpus)

19099

In [92]:
d19099 = pd.read_excel('d19099.xlsx',sheet_name='Sheet1')

In [170]:
doc_topics = []
for i in range(len(corpus)):
    topics = ldamodel.get_document_topics(corpus[i],minimum_probability=0.3)
    if topics != []:
        doc_topics.append(tuple(map(lambda topic:topic[0],topics)))
    else:
        doc_topics.append(())

In [171]:
doc_topics[5]

(0, 5)

In [172]:
d19099['topics'] = pd.Series(doc_topics)

In [173]:
d19099.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19099 entries, 0 to 19098
Data columns (total 15 columns):
연월        19099 non-null int64
전시제목      19099 non-null object
링크        19099 non-null object
전시작가      19099 non-null object
전시일정      19099 non-null object
초대일시      15524 non-null object
관람시간      19099 non-null object
전시장소      19099 non-null object
주소        19099 non-null object
연락처       19099 non-null object
홈페이지      15998 non-null object
아티클       19099 non-null object
텍스트       19099 non-null object
이미지       19099 non-null object
topics    19099 non-null object
dtypes: int64(1), object(14)
memory usage: 2.3+ MB


In [174]:
d19099.to_excel('data_final.xlsx')

In [86]:
ldamodel.show_topics(num_words=30) #토픽을 구성하는 주요단어들

[(0,
  '0.071*"사진" + 0.046*"도시" + 0.018*"풍경" + 0.015*"회화" + 0.011*"장소" + 0.010*"건물" + 0.010*"기록" + 0.009*"카메라" + 0.009*"촬영" + 0.007*"서울" + 0.006*"사진가" + 0.006*"장면" + 0.004*"건축" + 0.004*"포착" + 0.004*"시리즈" + 0.004*"건축물" + 0.004*"아파트" + 0.003*"거리" + 0.003*"역사" + 0.003*"환경" + 0.003*"현장" + 0.003*"밤" + 0.003*"집" + 0.003*"재현" + 0.003*"영화" + 0.003*"지역" + 0.003*"배경" + 0.003*"개발" + 0.003*"인물" + 0.003*"사진작가"'),
 (1,
  '0.005*"인식" + 0.005*"구조" + 0.004*"사유" + 0.004*"차원" + 0.004*"물질" + 0.004*"실재" + 0.004*"현상" + 0.003*"감각" + 0.003*"생" + 0.003*"우주" + 0.003*"본질" + 0.003*"기호" + 0.003*"주의" + 0.003*"영역" + 0.003*"추상" + 0.003*"재현" + 0.003*"행위" + 0.003*"차이" + 0.003*"경계" + 0.003*"질서" + 0.003*"무한" + 0.003*"지각" + 0.003*"주체" + 0.003*"상태" + 0.003*"표면" + 0.003*"전체" + 0.002*"언어" + 0.002*"관점" + 0.002*"비" + 0.002*"지점"'),
 (2,
  '0.009*"욕망" + 0.009*"기억" + 0.007*"인물" + 0.006*"그녀" + 0.006*"얼굴" + 0.005*"감정" + 0.005*"여성" + 0.005*"몸" + 0.005*"자기" + 0.004*"자아" + 0.004*"죽음" + 0.004*"불안" + 0.003*"사회" + 0.003*"현대인" + 0.003*"타인

#### 참고
https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/#11createthedictionaryandcorpusneededfortopicmodeling

In [22]:
# Compute Perplexity k=15:-8.692 , k=10: -8.696, k=7:-9.34359172454347(tf=2)
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.583079876972759


In [82]:
# Compute Coherence Score , k=10:0.527976169265899
coherence_model_lda = CoherenceModel(model=ldamodel, texts=nouns_refined, dictionary=dic, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4928798219349263


In [175]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

C:\Users\Chankoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.117564  0.175944       1        1  19.892765
2     -0.148630  0.003400       2        1  16.515442
8     -0.170728 -0.178622       3        1  11.267961
6     -0.143043 -0.042532       4        1  10.976214
5     -0.093638  0.075759       5        1   8.979726
7      0.258933 -0.003623       6        1   8.823459
3      0.179833 -0.060714       7        1   6.865264
9      0.122430  0.151317       8        1   6.108650
0      0.029081  0.090107       9        1   5.686636
4      0.083327 -0.211036      10        1   4.883884, topic_info=     Category          Freq Term         Total  loglift  logprob
term                                                            
504   Default  14154.000000   사진  14154.000000  30.0000  30.0000
101   Default   9184.000000   도시   9184.000000  29.0000  29.0000
596   Default   8147.000000    꽃   8147.000000  28.0000  28.0000
895   Default   6100.000000   조각   6100.000000  27.0000  27.0000
229   Default   5923.000000   풍경   5923.000000  26.0000  26.0000
26    Default   5537.000000  미술관   5537.000000  25.0000  25.0000
6559  Default   5051.000000   자연   5051.000000  24.0000  24.0000
3077  Default   4536.000000    색   4536.000000  23.0000  23.0000
2390  Default   3958.000000   한국   3958.000000  22.0000  22.0000
1565  Default   5403.000000    빛   5403.000000  21.0000  21.0000
2616  Default   3077.000000   회화   3077.000000  20.0000  20.0000
1323  Default   3954.000000   그림   3954.000000  19.0000  19.0000
133   Default   3864.000000   사물   3864.000000  18.0000  18.0000
258   Default   6213.000000   나무   6213.000000  17.0000  17.0000
166   Default   4192.000000   영상   4192.000000  16.0000  16.0000
733   Default   3425.000000   재료   3425.000000  15.0000  15.0000
87    Default   5802.000000   기억   5802.000000  14.0000  14.0000
870   Default   4961.000000   색채   4961.000000  13.0000  13.0000
1171  Default   4540.000000   아트   4540.000000  12.0000  12.0000
283   Default   3325.000000   아이   3325.000000  11.0000  11.0000
805   Default   4834.000000   장소   4834.000000  10.0000  10.0000
767   Default   3976.000000   년대   3976.000000   9.0000   9.0000
2787  Default   2875.000000   미술   2875.000000   8.0000   8.0000
409   Default   4173.000000   관객   4173.000000   7.0000   7.0000
645   Default   5927.000000   욕망   5927.000000   6.0000   6.0000
42    Default   2776.000000    월   2776.000000   5.0000   5.0000
34    Default   3946.000000   서울   3946.000000   4.0000   4.0000
1254  Default   4886.000000   화가   4886.000000   3.0000   3.0000
1566  Default   3815.000000   사랑   3815.000000   2.0000   2.0000
12    Default   3612.000000   기획   3612.000000   1.0000   1.0000
...       ...           ...  ...           ...      ...      ...
7330  Topic10    284.775208  강아지    325.644531   2.8851  -6.4014
5402  Topic10    380.837250  장난감    457.735504   2.8353  -6.1108
2256  Topic10    420.049072   유쾌    525.190002   2.7958  -6.0128
3613  Topic10    360.225464   패션    443.361420   2.8116  -6.1664
283   Topic10   2036.457153   아이   3325.787598   2.5287  -4.4342
1321  Topic10    214.845261   구매    245.073029   2.8876  -6.6832
2386  Topic10    293.974579    팝    370.882660   2.7868  -6.3697
7820  Topic10    322.281830   커피    419.088531   2.7566  -6.2777
1566  Topic10   1607.836792   사랑   3815.422852   2.1551  -4.6705
2048  Topic10    778.248291   동화   1475.459473   2.3796  -5.3961
261   Topic10    784.677368   놀이   1497.019043   2.3733  -5.3879
1980  Topic10   1366.213013   동물   3496.269531   2.0796  -4.8333
4026  Topic10    940.605286   가족   2110.035645   2.2113  -5.2066
3162  Topic10    945.736511  디자인   2390.719482   2.0918  -5.2012
3204  Topic10    695.300415   물건   1477.355713   2.2656  -5.5088
1127  Topic10    963.267334   대중   2501.253174   2.0650  -5.1828
7499  Topic10    390.825806   음식    606.726379   2.5794  -6.0849
871   Topic10    594.540222   소비   1307.949951   2.2308  -5

![1](https://user-images.githubusercontent.com/38183218/44544400-0ef41e00-a74d-11e8-894d-0260f2672cc9.PNG)

In [100]:
query = """빈센트 반 고흐의 대표작 중 하나로 꼽히는 <별이 빛나는 밤>은 그가 고갱과 다툰 뒤 자신의 귀를 자른 사건 이후 생레미의 요양원에 있을 때 그린 것이다. 반 고흐에게 밤하늘은 무한함을 표현하는 대상이었고, 이보다 먼저 제작된 아를의 <밤의 카페 테라스>나 <론 강 위로 별이 빛나는 밤>에서도 별이 반짝이는 밤의 정경을 다루었다. 반 고흐는 동생 테오에게 쓴 편지에서 “오늘 아침 나는 해가 뜨기 한참 전에 창문을 통해 아무것도 없고 아주 커 보이는 샛별밖에 없는 시골을 보았다.”고 했다. 이 샛별은 그림 가운데 왼쪽에 있는 커다란 흰 별일 것이다. 그가 그린 밤하늘에서는 구름과 대기, 별빛과 달빛이 폭발하고 있다. 황량하고 짙은 파란색 하늘은 세상의 종말을 연상케 하고, 그 위로는 구름이 소용돌이치며 떠있다. 달과 별의 둘레에는 뿌옇게 무리가 져있다.

비연속적이고 동적인 터치로 그려진 하늘은 굽이치는 두꺼운 붓놀림으로 불꽃같은 사이프러스와 연결되고, 그 아래의 마을은 대조적으로 평온하고 고요하다. 마을은 있는 그대로 그린 것이 아니라 부분적으로는 고안되었는데, 교회 첨탑은 반 고흐의 고향 네덜란드를 연상시킨다. 그는 병실 밖으로 내다보이는 밤 풍경을 기억과 상상을 결합시켜 그렸는데, 이는 자연에 대한 반 고흐의 내적이고 주관적인 표현을 구현하고 있다. 수직으로 높이 뻗어 땅과 하늘을 연결하는 사이프러스는 전통적으로 무덤이나 애도와 연관된 나무이지만, 반 고흐는 죽음을 불길하게 보지 않았다. 

그는 “별을 보는 것은 언제나 나를 꿈꾸게 한다.”면서 “왜 하늘의 빛나는 점들에는 프랑스 지도의 검은 점처럼 닿을 수 없을까? 타라스콩이나 루앙에 가려면 기차를 타듯이, 우리는 별에 다다르기 위해 죽는다.”고 했다. 이 시기에 그의 필치는 더욱 두꺼워지고 더욱 열정적으로 변했으며, 꿈틀거리는 듯한 선은 별의 광채를 한층 두드러지게 한다. 이 시기의 특징인 회오리치는 듯 꿈틀거리는 필치는 강렬한 색과 결합되어 감정을 더욱 격렬하게 표현한다. 이 곡선의 화필은 굽이치는 운동감을 표현하면서, 그림 전체를 율동적인 흐름으로 통합한다.
"""

In [101]:
tw = Twitter()

In [102]:
query_tokens = tw.nouns(query)

In [103]:
dic = corpora.Dictionary(nouns_refined)

In [106]:
len(dic)

62349

In [110]:
dic[3]

'계기'

In [104]:
query_bow = dic.doc2bow(query_tokens)

In [105]:
ldamodel.get_document_topics(query_bow)

[(0, 0.016855234),
 (1, 0.049830336),
 (2, 0.07705122),
 (3, 0.07372038),
 (5, 0.12893775),
 (6, 0.116763055),
 (7, 0.03887868),
 (8, 0.49688777)]

#### LDA_artists

In [13]:
# with open('artists_twice_morphs_final.lst','rb') as file:
#     artists_twice_tokens = pickle.load(file)

In [28]:
# with open('artists_twice_adnouns_final.lst','rb') as file:
#     artists_twice_tokens = pickle.load(file)

In [50]:
with open('artists_twice_nouns_final.lst','rb') as file:
    artists_twice_tokens = pickle.load(file)

In [53]:
with open('TFIDF_artists_nouns_final.dic','rb') as file:
    TFIDF = pickle.load(file)

In [55]:
TFIDF['백남준']

{'은': 0.6023647507690048,
 '한국': 0.4630128245852305,
 '작가': 0.04654905243844683,
 '인의': 1.0209166211873493,
 '특색': 2.1823465017700445,
 '그룹': 1.0377118470351678,
 '전': 0.2815729475455113,
 '월': 0.43959842412188804,
 '일': 0.23098886482987702,
 '개최': 0.5192061838080303,
 '일상': 0.3568695729361704,
 '내면': 0.6706553138040664,
 '이야기': 0.35691903217140464,
 '담': 1.4332963425067358,
 '작품': 0.08624650609618159,
 '김': 1.0049057900968847,
 '원숙': 2.5465838995530348,
 '자연': 0.3925216754481051,
 '동양': 1.223766195108884,
 '감성': 0.6146575073309796,
 '재': 0.49685671274473914,
 '노은': 4.202100913555818,
 '뉴욕': 1.6409507698765686,
 '변종': 2.7505703046584555,
 '곤': 2.3883838567004645,
 '과거': 0.6124427743419247,
 '시간': 0.25748176874906564,
 '기록': 0.7937727661197772,
 '이진용': 4.046795274046059,
 '화투': 4.165603944888314,
 '소재': 0.5446069702574562,
 '팝아트': 1.8629656987482892,
 '조영남': 3.5678269607001134,
 '선': 0.244639071308939,
 '또한': 0.2472251721875621,
 '비디오': 1.927460882806931,
 '아트': 0.7021148574058039,
 '선구

In [16]:
len(artists_twice_tokens)

8892

불용어처리 느낌으로 TFIDF 1 이하 단어는 제외한다

In [56]:
artists_twice_tokens_refined = []
for tpl in artists_twice_tokens:
    artist_name = tpl[0]
    tokens = tpl[1]
    
    tmp = []
    for token in tokens:
        if (TFIDF[artist_name][token] >1):
            tmp.append(token)
    artists_twice_tokens_refined.append(tmp)

In [57]:
dic = corpora.Dictionary(artists_twice_tokens_refined)

In [58]:
len(dic) #출현 단어 개수

51209

In [59]:
corpus = [dic.doc2bow(doc) for doc in artists_twice_tokens_refined] #단어별 출현 횟수 tuple 구성

In [65]:
%%time
ldamodel = gensim.models.ldamodel.LdaModel(
                        corpus, 
                        num_topics=7, 
                        id2word = dic, 
                        passes=10
                        ) #LDA model 생성

Wall time: 4min 40s


In [66]:
ldamodel.save('lda_artist_7.model')

In [67]:
ldamodel = gensim.models.ldamodel.LdaModel.load('lda_artist_7.model')

In [68]:
ldamodel.show_topics(num_words=30) #토픽을 구성하는 주요단어들

[(0,
  '0.009*"꽃" + 0.004*"부산" + 0.004*"산" + 0.004*"화가" + 0.003*"산수화" + 0.003*"바다" + 0.003*"땅" + 0.003*"산수" + 0.003*"수묵" + 0.003*"동양화" + 0.003*"화의" + 0.002*"소나무" + 0.002*"동양" + 0.002*"봄" + 0.002*"자화상" + 0.002*"바람" + 0.002*"기운" + 0.002*"민화" + 0.002*"옛" + 0.002*"조화" + 0.002*"붓" + 0.002*"토끼" + 0.002*"제주" + 0.002*"한지" + 0.002*"채색" + 0.002*"얼굴" + 0.002*"마을" + 0.001*"매화" + 0.001*"고향" + 0.001*"그린"'),
 (1,
  '0.005*"광주" + 0.004*"프로그램" + 0.004*"국제" + 0.004*"미디어" + 0.003*"지역" + 0.003*"양평" + 0.003*"백남준" + 0.003*"인천" + 0.003*"교류" + 0.003*"아시아" + 0.003*"퍼포먼스" + 0.003*"국내" + 0.003*"센터" + 0.003*"시립" + 0.003*"비디오" + 0.003*"행사" + 0.003*"아티스트" + 0.002*"비엔날레" + 0.002*"시민" + 0.002*"여성" + 0.002*"레" + 0.002*"그룹" + 0.002*"일본" + 0.002*"경기도" + 0.002*"스튜디오" + 0.002*"영국" + 0.002*"개관" + 0.002*"플랫폼" + 0.002*"교육" + 0.002*"국내외"'),
 (2,
  '0.017*"조각" + 0.009*"대전" + 0.008*"대구" + 0.007*"년대" + 0.007*"화풍" + 0.006*"추상" + 0.006*"화가" + 0.005*"분단" + 0.004*"낙우" + 0.004*"서양화" + 0.004*"근" + 0.004*"구상" + 0.004*"시기" + 0.004*"당시" 

In [31]:
# Compute Perplexity 
print('\nPerplexity: ', ldamodel.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


Perplexity:  -8.77546636425197


In [33]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=ldamodel, texts=artists_twice_tokens_refined, dictionary=dic, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.4610406153790064


In [69]:
# Visualize the topics

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(ldamodel, corpus, dic)
vis

C:\Users\Chankoo\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6      0.105574 -0.010419       1        1  30.988153
1     -0.128190 -0.130316       2        1  14.062784
5      0.157270  0.035361       3        1  13.210842
3      0.030073  0.008187       4        1  11.395717
4      0.028967 -0.127399       5        1  10.969367
0      0.032143  0.127908       6        1  10.216621
2     -0.225837  0.096679       7        1   9.156513, topic_info=      Category         Freq  Term        Total  loglift  logprob
term                                                            
28506  Default  8616.000000    조각  8616.000000  30.0000  30.0000
7485   Default  5326.000000     꽃  5326.000000  29.0000  29.0000
865    Default  6111.000000    대전  6111.000000  28.0000  28.0000
832    Default  5904.000000    대구  5904.000000  27.0000  27.0000
6413   Default  5119.000000    년대  5119.000000  26.0000  26.0000
21171  Default  7166.000000    도시  7166.000000  25.0000  25.0000
19144  Default  3301.000000    추상  3301.000000  24.0000  24.0000
7360   Default  5740.000000    화가  5740.000000  23.0000  23.0000
4376   Default  3798.000000    화풍  3798.000000  22.0000  22.0000
961    Default  6555.000000    동물  6555.000000  21.0000  21.0000
289    Default  4624.000000    공예  4624.000000  20.0000  20.0000
28616  Default  3580.000000    나무  3580.000000  19.0000  19.0000
337    Default  4876.000000    광주  4876.000000  18.0000  18.0000
5130   Default  2644.000000    분단  2644.000000  17.0000  17.0000
1006   Default  5671.000000   디자인  5671.000000  16.0000  16.0000
17523  Default  2522.000000     빛  2522.000000  15.0000  15.0000
24209  Default  2460.000000    욕망  2460.000000  14.0000  14.0000
14088  Default  2208.000000    낙우  2208.000000  13.0000  13.0000
27270  Default  2475.000000    물질  2475.000000  12.0000  12.0000
30250  Default  2632.000000    지역  2632.000000  11.0000  11.0000
2453   Default  2557.000000    양평  2557.000000  10.0000  10.0000
4131   Default  6098.000000  프로그램  6098.000000   9.0000   9.0000
29334  Default  2233.000000     색  2233.000000   8.0000   8.0000
1677   Default  4105.000000    부산  4105.000000   7.0000   7.0000
4026   Default  5730.000000    판화  5730.000000   6.0000   6.0000
2364   Default  5861.000000    아이  5861.000000   5.0000   5.0000
947    Default  4062.000000     돌  4062.000000   4.0000   4.0000
1549   Default  2708.000000   백남준  2708.000000   3.0000   3.0000
1798   Default  5581.000000    사랑  5581.000000   2.0000   2.0000
442    Default  2816.000000     근  2816.000000   1.0000   1.0000
...        ...          ...   ...          ...      ...      ...
6692    Topic7   950.632629    비구  1117.280151   2.2292  -6.2846
832     Topic7  4084.797119    대구  5904.664551   2.0222  -4.8267
6413    Topic7  3512.039062    년대  5119.747070   2.0138  -4.9777
3211    Topic7   527.058899   장욱진   586.722168   2.2835  -6.8744
2170    Topic7   861.561462    스승  1039.365112   2.2031  -6.3829
442     Topic7  1979.887085     근  2816.075928   2.0384  -5.5509
1963    Topic7   745.468018    선언   898.033386   2.2045  -6.5277
3403    Topic7   877.014282    제자  1151.099609   2.1188  -6.3652
4357    Topic7  1708.212036    화단  2752.241211   1.9137  -5.6985
172     Topic7   815.047241    결성  1064.117432   2.1240  -6.4384
7360    Topic7  2833.273438    화가  5740.056641   1.6847  -5.1925
1940    Topic7  2073.838623   서양화  3769.691406   1.7931  -5.5045
750     Topic7   717.905823    누드   900.164429   2.1645  -6.5654
2440    Topic7   676.687012    양국   851.848938   2.1605  -6.6245
3296    Topic7  1806.725708    전쟁  4041.487305   1.5856  -5.6424
2208    Topic7  1923.272827    시기  4620.215820   1.5143  -5.5799
445     Topic7  1720.504395    근대  3885.300537   1.5761  -5.6913
1962    Topic7  1129.719849    선생  1971.880859   1.8337  -6.1120
365     Topic7  1941.866089    구상  5140.800293   1.4171  -5.5703
827     Topic7  1850.089600    당시  4888.716309   1.4190  -5.6187
2935    Topic7  1144.55969